<a href="https://colab.research.google.com/github/vishnusnath447/test-google-colab/blob/main/WebPageSummerizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers requests beautifulsoup4

In [4]:
import requests
from bs4 import BeautifulSoup
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

# Function to fetch and scrape reviews from the webpage
def scrape_reviews(url):
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # Adjust based on your actual HTML structure
    reviews = []
    review_elements = soup.find_all('p')
    for review in review_elements:
        reviews.append(review.get_text(strip=True))

    return reviews

# Function to generate suggestions based on the summaries
def generate_suggestions(summary):
    # Implement your suggestion generation logic here
    # For example, extract common themes or issues from the summary
    suggestions = "Placeholder suggestion: Improve battery life and add dark mode."
    return suggestions

# Main function to scrape, summarize, and generate suggestions
def main():
    url = "https://1ddd7f45-3272-4e73-8105-c15e6e7560c4-00-2twkoe64h1tyw.sisko.replit.dev/"  # Replace with your actual webpage URL
    reviews = scrape_reviews(url)

    if not reviews:
        print("No reviews found. Please check the class name and the webpage structure.")
        return

    # Initialize the Pegasus tokenizer and model
    model_name = "google/pegasus-large"
    tokenizer = PegasusTokenizer.from_pretrained(model_name)
    model = PegasusForConditionalGeneration.from_pretrained(model_name).cuda()  # Move model to GPU

    # Summarize all reviews
    combined_reviews = " ".join(reviews)
    inputs = tokenizer.encode("summarize: " + combined_reviews, return_tensors="pt", max_length=1024, truncation=True).cuda()  # Move input to GPU
    summary_ids = model.generate(inputs, max_length=150, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # Generate suggestions based on the summary
    suggestions = generate_suggestions(summary)

    # Print the combined summary and suggestions
    print("Combined Summary of Reviews:")
    print(summary)
    print("\nCommon Suggestions:")
    print(suggestions)

if __name__ == "__main__":
    main()


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Combined Summary of Reviews:
The interface is intuitive, but I really wish there was a dark mode option for better night viewing. Love the app's functionality, but a dark mode and calendar integration would make it perfect!

Common Suggestions:
Placeholder suggestion: Improve battery life and add dark mode.


In [ ]:
import requests
from bs4 import BeautifulSoup
from transformers import pipeline

# Function to fetch and scrape reviews from the webpage
def scrape_reviews(url):
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # Update the class name based on your webpage structure
    review_elements = soup.find_all(class_='reviews')

    reviews = [review.get_text(strip=True) for review in review_elements]
    return reviews

# Function to summarize the reviews
def summarize_reviews(reviews, summarizer):
    summaries = [summarizer(review, max_length=150, min_length=30, do_sample=False)[0]['summary_text'] for review in reviews]
    return summaries

# Function to generate suggestions based on the summaries
def generate_suggestions(summaries, generator):
    combined_summaries = " ".join(summaries)
    prompt = f"Based on the following reviews, provide suggestions for improvement: {combined_summaries}"

    suggestions = generator(prompt, max_length=150, num_return_sequences=1)
    return suggestions[0]['generated_text']

# Main function to scrape, summarize, and generate suggestions
def main():
    url = "https://1ddd7f45-3272-4e73-8105-c15e6e7560c4-00-2twkoe64h1tyw.sisko.replit.dev/"  # Replace with your actual webpage URL
    reviews = scrape_reviews(url)

    if not reviews:
        print("No reviews found. Please check the class name and the webpage structure.")
        return

    # Initialize the summarizer
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

    # Summarize the reviews
    summaries = summarize_reviews(reviews, summarizer)

    # Initialize the text generator
    generator = pipeline("text-generation", model="facebook/bart-large-cnn")

    # Generate suggestions based on summaries
    suggestions = generate_suggestions(summaries, generator)

    # Print summaries and suggestions
    print("Summaries:")
    for i, summary in enumerate(summaries, 1):
        print(f"Review {i}: {summary}\n")

    print("Suggestions:")
    print(suggestions)

if __name__ == "__main__":
    main()


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Summaries:
Review 1: The battery drain is unreal! My phone barely lasts a day. Wish there was a battery saver mode. It would be amazing if you could integrate a calendar sync feature. The interface is intuitive, but I really wish there was an option for better night viewing.

Suggestions:
Based on the following reviews, provide suggestions for improvement: The battery drain is unreal! My phone barely lasts a day. Wish there was a battery saver mode. It would be amazing if you could integrate a calendar sync feature. The interface is intuitive, but I really wish there was an option for better night viewing. over over onete one over their overte one is one over l over over over just one over one over Their overte last with over overge over still overge just one isara one over still one over with one over my overge one over is onease over stillge over just overge which with their their over their one over"ge over over a overge like overge a whichge overge between with


In [1]:
import requests
from bs4 import BeautifulSoup
from transformers import T5ForConditionalGeneration, T5Tokenizer, pipeline

# Function to fetch and scrape reviews from the webpage
def scrape_reviews(url):
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # Update the class name based on your webpage structure
    # review_elements = soup.find_all(class_='reviews')

    # reviews = [review.get_text(strip=True) for review in review_elements]
    # return reviews
    reviews = []
    review_elements = soup.find_all('p')
    for review in review_elements:
        reviews.append(review.get_text(strip=True))

    return reviews

# Function to summarize the reviews
def summarize_reviews(reviews, summarizer):
    summaries = [summarizer(review, max_length=150, min_length=30, do_sample=False)[0]['summary_text'] for review in reviews]
    return summaries

# Function to generate suggestions based on the summaries
def generate_suggestions(summaries, generator, tokenizer):
    combined_summaries = " ".join(summaries)
    prompt = f"Based on the following reviews, provide suggestions for improvement: {combined_summaries}"

    inputs = tokenizer.encode("summarize: " + prompt, return_tensors="pt", max_length=512, truncation=True)
    outputs = generator.generate(inputs, max_length=150, num_beams=5, early_stopping=True)
    suggestions = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return suggestions

# Main function to scrape, summarize, and generate suggestions
def main():
    url = "https://1ddd7f45-3272-4e73-8105-c15e6e7560c4-00-2twkoe64h1tyw.sisko.replit.dev/"  # Replace with your actual webpage URL
    reviews = scrape_reviews(url)

    if not reviews:
        print("No reviews found. Please check the class name and the webpage structure.")
        return

    # Initialize the summarizer
    summarizer = pipeline("summarization", model="t5-large")

    # Summarize the reviews
    summaries = summarize_reviews(reviews, summarizer)

    # Initialize the T5 model for text generation
    model_name = "t5-large"
    generator = T5ForConditionalGeneration.from_pretrained(model_name)
    tokenizer = T5Tokenizer.from_pretrained(model_name)

    # Generate suggestions based on summaries
    suggestions = generate_suggestions(summaries, generator, tokenizer)

    # Print summaries and suggestions
    print("Summaries:")
    for i, summary in enumerate(summaries, 1):
        print(f"Review {i}: {summary}\n")

    print("Suggestions:")
    print(suggestions)

if __name__ == "__main__":
    main()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Your max_length is set to 150, but your input_length is only 27. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)
Your max_length is set to 150, but your input_length is only 20. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...'

Summaries:
Review 1: battery drain is unreal! my phone barely lasts a day . love the features of this app, but the battery drain .

Review 2: great app concept, but the constant background refresh kills my battery . please optimize the background refresh rate . i'm using a nexus 6 tablet and it's killing my battery!

Review 3: this app is awesome, but the battery life suffers after recent updates . wish there was a battery saver mode, if there was one .

Review 4: the interface is intuitive, but i really wish there was a dark mode option for better night viewing . it's a shame there isn't a darker mode option, as it would have been more useful at night .

Review 5: calendar sync feature would be great . i'm using it on my iphone 6s plus ipad mini . would be awesome if you could integrate it with google calendar.

Review 6: a dark mode and calendar integration would make it perfect . the app has a lot of great features, but it could use a little more polish .

Suggestions:
based on the 

In [ ]:
import requests
from bs4 import BeautifulSoup
from transformers import T5ForConditionalGeneration, T5Tokenizer, pipeline

# Function to fetch and scrape reviews from the webpage
def scrape_reviews(url):
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # Adjust based on your actual HTML structure
    reviews = []
    review_elements = soup.find_all('p')
    for review in review_elements:
        reviews.append(review.get_text(strip=True))

    return reviews

# Function to summarize the reviews
def summarize_reviews(reviews, summarizer):
    combined_reviews = " ".join(reviews)
    summary = summarizer(combined_reviews, max_length=150, min_length=30, do_sample=False)
    return summary[0]['summary_text']

# Function to generate suggestions based on the summaries
def generate_suggestions(summary, generator, tokenizer):
    prompt = f"Based on the following reviews, provide suggestions for improvement: {summary}"

    inputs = tokenizer.encode(prompt, return_tensors="pt", max_length=512, truncation=True)
    outputs = generator.generate(inputs, max_length=150, num_beams=5, early_stopping=True)
    suggestions = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return suggestions

# Main function to scrape, summarize, and generate suggestions
def main():
    url = "https://1ddd7f45-3272-4e73-8105-c15e6e7560c4-00-2twkoe64h1tyw.sisko.replit.dev/"  # Replace with your actual webpage URL
    reviews = scrape_reviews(url)

    if not reviews:
        print("No reviews found. Please check the class name and the webpage structure.")
        return

    # Initialize the summarizer
    summarizer = pipeline("summarization", model="t5-large")

    # Summarize all reviews
    summary = summarize_reviews(reviews, summarizer)

    # Initialize the T5 model for text generation
    model_name = "t5-large"
    generator = T5ForConditionalGeneration.from_pretrained(model_name)
    tokenizer = T5Tokenizer.from_pretrained(model_name)

    # Generate suggestions based on the summary
    suggestions = generate_suggestions(summary, generator, tokenizer)

    # Print the combined summary and suggestions
    print("Combined Summary of Reviews:")
    print(summary)
    print("\nCommon Suggestions:")
    print(suggestions)

if __name__ == "__main__":
    main()


Your max_length is set to 150, but your input_length is only 133. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=66)
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Combined Summary of Reviews:
this app is awesome, but the battery life suffers after recent updates. wish there was a battery saver mode. the interface is intuitive, but a dark mode option for better night viewing would be perfect.

Common Suggestions:
app is awesome, but the battery life suffers after recent updates. but the battery life suffers after recent updates. awesome, but the battery life suffers after recent updates. awesome, but the battery life suffers after recent updates. Based on the following reviews, provide suggestions for improvement: provide suggestions: provide suggestions for provide suggestions for improvement: the and and............


In [ ]:
import requests
from bs4 import BeautifulSoup
from transformers import T5ForConditionalGeneration, T5Tokenizer, pipeline

# Function to fetch and scrape reviews from the webpage
def scrape_reviews(url):
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # Adjust based on your actual HTML structure
    reviews = []
    review_elements = soup.find_all('p')
    for review in review_elements:
        reviews.append(review.get_text(strip=True))

    return reviews

# Function to summarize the reviews
def summarize_reviews(reviews, summarizer):
    combined_reviews = " ".join(reviews)
    input_length = len(combined_reviews)
    max_length = min(input_length // 2, 150)  # Adjust max_length dynamically
    summary = summarizer(combined_reviews, max_length=max_length, min_length=20, do_sample=False)
    return summary[0]['summary_text']

# Function to generate suggestions based on the summaries
def generate_suggestions(summary, generator, tokenizer):
    prompt = f"Based on the following reviews, provide suggestions for improvement: {summary}"

    inputs = tokenizer.encode(prompt, return_tensors="pt", max_length=512, truncation=True)
    outputs = generator.generate(inputs.cuda(), max_length=150, num_beams=5, early_stopping=True)
    suggestions = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return suggestions

# Main function to scrape, summarize, and generate suggestions
def main():
    url = "https://1ddd7f45-3272-4e73-8105-c15e6e7560c4-00-2twkoe64h1tyw.sisko.replit.dev/"  # Replace with your actual webpage URL
    reviews = scrape_reviews(url)

    if not reviews:
        print("No reviews found. Please check the class name and the webpage structure.")
        return

    # Initialize the summarizer
    summarizer = pipeline("summarization", model="t5-large", device=0)  # Utilize GPU explicitly

    # Summarize all reviews
    summary = summarize_reviews(reviews, summarizer)

    # Initialize the T5 model for text generation
    model_name = "t5-large"
    generator = T5ForConditionalGeneration.from_pretrained(model_name).cuda()  # Move model to GPU
    tokenizer = T5Tokenizer.from_pretrained(model_name)

    # Generate suggestions based on the summary
    suggestions = generate_suggestions(summary, generator, tokenizer)

    # Print the combined summary and suggestions
    print("Combined Summary of Reviews:")
    print(summary)
    print("\nCommon Suggestions:")
    print(suggestions)

if __name__ == "__main__":
    main()


Your max_length is set to 150, but your input_length is only 133. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=66)
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Combined Summary of Reviews:
this app is awesome, but the battery life suffers after recent updates. wish there was a battery saver mode. the interface is intuitive, but a dark mode option for better night viewing would be perfect.

Common Suggestions:
app is awesome, but the battery life suffers after recent updates. but the battery life suffers after recent updates. awesome, but the battery life suffers after recent updates. awesome, but the battery life suffers after recent updates. Based on the following reviews, provide suggestions for improvement: provide suggestions: provide suggestions for provide suggestions for improvement: the and and............
